# Experimenting with MLModels
---------------

Loading data and models for MLModels, cross validations

In [1]:
import __init__
import models.cross_validator as cross_validator
import models.datahandler as datahandler
from models.datamodel import DataModel
import models.outputmaker as outputmaker
import evaluator.metrics as metrics
from models.mlmodels import MLModels

trial = datahandler.load_train('../data/dataset/tsd_trial.csv', verbose=True)
train = datahandler.load_train('../data/dataset/tsd_train.csv', verbose=True)

import warnings
warnings.filterwarnings("ignore")

/home/hb/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."


Initialize `MLModels` Methods

In [2]:
model = MLModels()

datamodel = DataModel(model='crf')

output_maker = outputmaker.crf_output

evaluator = metrics

### Cross Validations

In [6]:
print("TRIAL DATA:")
data = trial
cv=5
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

cv=10
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

0it [00:00, ?it/s]

TRIAL DATA:


5it [03:30, 42.12s/it]
0it [00:00, ?it/s]

---------------------------------------------------------------------------
5-Fold Cross Validation Averaged Results:
F1 = 0.56 ± 0.01, 	 P = 0.64 ± 0.01, 	 R = 0.56 ± 0.02
---------------------------------------------------------------------------


10it [07:21, 44.13s/it]

---------------------------------------------------------------------------
10-Fold Cross Validation Averaged Results:
F1 = 0.56 ± 0.02, 	 P = 0.65 ± 0.02, 	 R = 0.56 ± 0.02
---------------------------------------------------------------------------


In [4]:
print("TRAIN DATA:")
data = train
cv=5
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

cv=10
cross_validator.cross_validate(train=data, datamodel=datamodel, model=model, evaluator=evaluator, 
                               output_maker=output_maker, cv=cv, print_results=False, dataset_logger=False,
                               calculate_average=True, return_folds=False)

0it [00:00, ?it/s]

TRAIN DATA:


5it [14:09, 169.85s/it]
0it [00:00, ?it/s]

---------------------------------------------------------------------------
5-Fold Cross Validation Averaged Results:
F1 = 0.54 ± 0.01, 	 P = 0.62 ± 0.01, 	 R = 0.54 ± 0.01
---------------------------------------------------------------------------


10it [21:10, 127.06s/it]

---------------------------------------------------------------------------
10-Fold Cross Validation Averaged Results:
F1 = 0.54 ± 0.01, 	 P = 0.62 ± 0.01, 	 R = 0.54 ± 0.01
---------------------------------------------------------------------------


### Train on train data, test on trial data

In [2]:
model = MLModels()

datamodel = DataModel(model='crf')

output_maker = outputmaker.crf_output

evaluator = metrics

In [3]:
X_train, y_train , _, _ = cross_validator.load_data(train, [i for i in range(train.shape[0])], 
                                                    datamodel, logger=True)

100%|██████████| 7939/7939 [02:10<00:00, 60.64it/s]


In [4]:
X_test, y_test , _, test_texts = cross_validator.load_data(trial, [i for i in range(trial.shape[0])], 
                                                           datamodel, logger=True)

100%|██████████| 690/690 [00:11<00:00, 57.72it/s]


In [5]:
model.fit(X_train, y_train)

preds = model.predict(X_test)

predictions = output_maker(X_test, test_texts, preds)

f1, p, r = evaluator.evaluate(gold = y_test, predictions = predictions)

print("F1:{}, \t P:{}, \t R:{}".format(f1, p, r))

F1:0.5941496976688349, 	 P:0.695784759554412, 	 R:0.5904613395863839
